# Second notebook - upload file df_new_GitHib

In [ ]:
import pandas as pd

df = pd.read_csv('df_new_GitHub.csv')

In [ ]:
# To have an overview of the data

summary = pd.DataFrame({
    'non_null_count': df.notnull().sum(),
    'null_percent': df.isnull().mean() * 100,
    'num_unique': df.nunique(),
    'data_type': df.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)


# Data cleansing - continue from the first file

In [ ]:
# Remove 'production_length' - no need in this analysis
df = df.drop(columns=['production_length'])

# Last_year, first_year

In [ ]:
# Explore first and last_year contant - Any 0 or -1 will becoume 'knownown'
import pandas as pd
import numpy as np

cols = ['first_year', 'last_year']

for col in cols:
    df[col] = df[col].apply(lambda x: "Unknown" if pd.isna(x) or x in [-1, 0, -1.0, 0.0] else str(int(x)))

    num_unknown = (df[col] == "Unknown").sum()
    percent_unknown = num_unknown / len(df) * 100
    print(f"[{col}] 'Unknown' values after conversion: {num_unknown} ({percent_unknown:.2f}%)")

# test columns
print("\nData types after conversion:")
print(df[cols].dtypes)


In [ ]:
# Check there are no 0 or -1
cols = ['first_year', 'last_year']

for col in cols:
    num_neg1 = (df[col] == -1).sum()
    print(f"[{col}] Number of -1 values after conversion: {num_neg1}")


In [ ]:
# for future analysis - change unknown into NaN which is float64 or int64
import pandas as pd
import numpy as np

cols = ['first_year', 'last_year']

for col in cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].replace([-1, 0], np.nan)

    # Statistics to check
    num_missing = df[col].isna().sum()
    percent_missing = num_missing / len(df) * 100
    print(f"[{col}] Missing/Unknown values after conversion: {num_missing} ({percent_missing:.2f}%)")
    print(f"[{col}] Data type after conversion: {df[col].dtype}")

# check
print("\nSample data:")
print(df[cols].head())


In [ ]:
# Check high years to detect outlier years
# Count table of values ​​in column first_year

counts = df['first_year'].value_counts()

# 10 highest values
top_10 = counts.nlargest(10)

# 10 lowest values
bottom_10 = counts.nsmallest(10)

print("🔹 10 הערכים הגבוהים ביותר:")
print(top_10)
print("\n🔹 10 הערכים הנמוכים ביותר:")
print(bottom_10)

In [ ]:
# Check 'first_year', 'last_year' columns after transformation
ols = ['first_year', 'last_year']

for col in cols:
    print(f"\n[{col}] unique problematic values before conversion:")
    print(df[col].value_counts(dropna=False).head(10))

In [ ]:
# Outliers years need to be removed
# Outliers years to remove

remove_years = [2025, 2026, 2029, 2046]

# Create a new DataFrame without the rows with these years
df = df[~df['first_year'].isin(remove_years)]

# check
print("✅ השורות עם השנים החריגות הוסרו.")
print(df['first_year'].value_counts().sort_index())

In [ ]:
# Check transformation
# Allow all columns to be displayed

pd.set_option('display.max_columns', None)

# List of unrealistic years
unrealistic_years = [2025, 2029, 2046, 1917, 1921, 1936, 1938, 1939, 1940, 1941, 1942, 1944, 1945]

# Displays all rows that match the unrealistic years
print(df[df['first_year'].isin(unrealistic_years)])

In [ ]:
# Check transformation
# All unique values ​​in column first_year

unique_years = df['first_year'].unique()

# Set aside the value -1 if it exists
unique_years = unique_years[unique_years != -1]

# Top 10 years
top_10_years = np.sort(unique_years)[-10:]

# Bottom 10 years
bottom_10_years = np.sort(unique_years)[:10]

print("🔹 10 השנים הגבוהות ביותר:", top_10_years)
print("🔹 10 השנים הנמוכות ביותר:", bottom_10_years)

# OUTLIERS WITH DATA CLEANSING

### number_of_seasons

In [ ]:
# Explore uniques and distribution in 'number_of_seasons'
df['number_of_seasons'].unique()


In [ ]:
# More effective display
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 1️⃣ Enable full display of all rows in output
pd.set_option('display.max_rows', None)

# 2️⃣ Full frequency table in order of values
frequency_table = df['number_of_seasons'].value_counts().sort_index()
print("Full frequency table:")
print(frequency_table)

# 3️⃣ List of all unique values
unique_values = df['number_of_seasons'].unique().tolist()
print("\nList of unique values:")
print(unique_values)

# 4️⃣ Bar graph showing the distribution
plt.figure(figsize=(12,5))
sns.barplot(x=frequency_table.index, y=frequency_table.values, palette="viridis")
plt.xlabel("Number of Seasons")
plt.ylabel("Count")
plt.title("Distribution of Number of Seasons")
plt.show()

# 5️⃣ Reset the option to display rows afterwards (optional)
pd.reset_option('display.max_rows')


In [ ]:
# The smallest values ​​in 'number_of_seasons'
# 5 smallest values

min_5_values = df['number_of_seasons'].nsmallest(5).unique()
min_5_rows = df[df['number_of_seasons'].isin(min_5_values)]
print("Rows with 5 smallest values of number_of_seasons:")
print(min_5_rows)

# 5 greatest values
max_10_values = df['number_of_seasons'].nlargest(10).unique()
max_10_rows = df[df['number_of_seasons'].isin(max_10_values)]
print("\nRows with 10 largest values of number_of_seasons:")
print(max_10_rows)


In [ ]:
# Data unknown and 0, -1 completion: Create new clean column 'number_of_seasons_clean'
# Data completion by other columns such as
# Your idea is to try to predict or "complete" the number_of_seasons value when it is unknown, based on other data we have such as:
# number_of_episodes → number of episodes
# first_year / last_year → years of broadcast
# Maybe also episode_run_time, genres, type, etc.

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# Create a copy of the original column
df['number_of_seasons_clean'] = df['number_of_seasons'].copy()

# 1️⃣ Convert values ​​0 and -1 to NaN (unknown)
df['number_of_seasons_clean'] = df['number_of_seasons_clean'].replace([0, -1], np.nan)

# 2️⃣ Prepare variables for the model (numeric features only)
features = ['number_of_episodes', 'episode_run_time', 'first_year', 'last_year']
df_model = df[features + ['number_of_seasons_clean']].copy()

# Keep known values ​​and separate unknown
train_data = df_model[df_model['number_of_seasons_clean'].notna()]
predict_data = df_model[df_model['number_of_seasons_clean'].isna()]

X_train = train_data[features]
y_train = train_data['number_of_seasons_clean']

X_predict = predict_data[features]

# 3️⃣ Handle missing values ​​in features (NaN, -1 or 0)
# First convert 0 and -1 in features For NaN
X_train = X_train.replace([0, -1], np.nan)
X_predict = X_predict.replace([0, -1], np.nan)

# Fill the NaN with the median of each column
X_train = X_train.fillna(X_train.median())
X_predict = X_predict.fillna(X_train.median())

# 4️⃣ Create a regression model
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# 5️⃣ Predict for the unknown only
predicted_seasons = model.predict(X_predict)

# 6️⃣ Replace the values ​​in the original df only for the unknown
df.loc[df['number_of_seasons_clean'].isna(), 'number_of_seasons_clean'] = np.round(predicted_seasons).astype(int)

# 7️⃣ Check - How many values ​​are filled
print("Number of values filled:", len(predicted_seasons))
print(df['number_of_seasons_clean'].value_counts().sort_index())



In [ ]:
# Check the data after transformation
summary = pd.DataFrame({
    'non_null_count': df.notnull().sum(),
    'null_percent': df.isnull().mean() * 100,
    'num_unique': df.nunique(),
    'data_type': df.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
# After transformation - look at data
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assume you have the column number_of_seasons_clean

# Filter values ​​up to 10 (including NaN)
seasons_cleaned = df['number_of_seasons_clean']
seasons_cleaned_filtered = seasons_cleaned[ (seasons_cleaned <= 10) | (seasons_cleaned.isna()) ]

# Count the number of occurrences of each unique value including NaN
counts = seasons_cleaned_filtered.value_counts(dropna=False).sort_index(key=lambda x: [float('-inf') if pd.isna(v) else v for v in x])

# Bar chart
plt.figure(figsize=(12,6))
plt.bar(counts.index.astype(str), counts.values, color='skyblue')  # הפיכה ל-string כדי שהNaN יוצג
plt.xlabel('Number of Seasons')
plt.ylabel('Count')
plt.title('Distribution of Number of Seasons (up to 10, including NaN)')
plt.xticks(rotation=45)
plt.show()

# Create a UNIQUE table with counts including NaN
unique_table_seasons = pd.DataFrame({
    'Number_of_Seasons': counts.index,
    'Count': counts.values
})

# Display the full table
print(unique_table_seasons)


In [ ]:
df['number_of_seasons_clean'].unique()

In [ ]:
# Check for NaN in new column 'number_of_seasons_clean'
# Check if they remain unknown (NaN)

num_unknown = df['number_of_seasons_clean'].isna().sum()
print("Number of remaining unknown values:", num_unknown)

if num_unknown > 0:
    print("There are still unknown values remaining.")
else:
    print("All unknown values have been filled.")


In [ ]:
# IQR on 'number_of_seasons_clean'
# Calculating Q1, Q3 and IQR

Q1 = df['number_of_seasons_clean'].quantile(0.25)
Q3 = df['number_of_seasons_clean'].quantile(0.75)
IQR = Q3 - Q1

# Outliers limits
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")
print(f"Lower bound = {lower_bound}, Upper bound = {upper_bound}")

# Marking outliers **temporarily** without adding a new column
outliers = df[(df['number_of_seasons_clean'] < lower_bound) | (df['number_of_seasons_clean'] > upper_bound)]

# Summarize how many rows are marked as Outlier
print(f"Number of outliers: {len(outliers)}")


In [ ]:
# quantile(0.95) - check values ​​below to remove qutlires later
# Choose the upper bound for the 95th percentile

upper_percentile = 0.95
upper_bound = df['number_of_seasons_clean'].quantile(upper_percentile)
print(f"95th percentile upper bound = {upper_bound}")

# Filter Outliers **temporarily**, without creating a new column
outliers = df[df['number_of_seasons_clean'] > upper_bound]

# Summarize how many rows are filtered as Outliers
print(f"Number of outliers (95th percentile): {len(outliers)}")



In [ ]:
# graph of 'number_of_seasons_clean' outliers
import matplotlib.pyplot as plt
import seaborn as sns

# Sample for a quick graph
sample_df = df.sample(10000, random_state=42)

# Outlier calculation according to the 95th percentile of number_of_seasons_clean
upper_bound = df['number_of_seasons_clean'].quantile(0.95)
sample_df['is_outlier'] = sample_df['number_of_seasons_clean'] > upper_bound

plt.figure(figsize=(10,6))

sns.scatterplot(
    data=sample_df,
    x='number_of_seasons_clean',
    y='popularity',
    hue='is_outlier',
    palette={False:'blue', True:'red'},
    alpha=0.6
)

plt.xlabel('Number of Seasons')
plt.ylabel('Popularity')
plt.title('Popularity vs Number of Seasons (Outliers >95th percentile in red)')
plt.legend(title='Outlier (>95th percentile)')
plt.show()


In [ ]:
df.info()

In [ ]:
# remove outliers here!!!!!!!!!!!
# remove 95% top outliers number_of_seasons_clean !!!!!!!!!
# Upper bound for 95th percentile

upper_percentile = 0.95
upper_bound = df['number_of_seasons_clean'].quantile(upper_percentile)
print(f"95th percentile upper bound = {upper_bound}")

# Keep only rows that are not Outliers
df = df[df['number_of_seasons_clean'] <= upper_bound]

# Check how many rows are left
print(f"Number of rows after removing outliers: {len(df)}")


In [ ]:
# Remove original 'number_of_seasons' column - no need for it anymore

df = df.drop(columns=['number_of_seasons'])

### number_of_episodes

In [ ]:
# 'number_of_episodes' uniques explore

import pandas as pd

# 1️⃣ Full frequency table for number_of_episodes
pd.set_option('display.max_rows', None)
frequency_table_episodes = df['number_of_episodes'].value_counts().sort_index()
print("Full frequency table for number_of_episodes:")
print(frequency_table_episodes)

# 2️⃣ List of all unique values
unique_values_episodes = df['number_of_episodes'].unique().tolist()
print("\nList of unique values for number_of_episodes:")
print(unique_values_episodes)

# 3️⃣ Reset the option to display rows afterwards (optional)
pd.reset_option('display.max_rows')


In [ ]:
# Data unknown and 0, -1 completion: Create new clean column 'number_of_episodes_clean'


import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

# ----- Step 1: Work on a copy to avoid damaging the original DF -----
df_work = df.copy()

# ----- Step 2: Create a clean column with unknown -----
df_work['number_of_episodes_clean'] = df_work['number_of_episodes'].replace(0, np.nan)

# ----- Step 3: Choose variables to predict -----
features = ['number_of_seasons_clean', 'episode_run_time', 'first_year', 'last_year']

# ----- Step 4: Handle missing values ​​in features temporarily -----
df_features = df_work[features].replace(-1, np.nan)

# ----- Step 5: Identify rows with unknown -----
predict_indexes = df_work[df_work['number_of_episodes_clean'].isna()].index

# Predict only where there is at least some information
valid_predict_indexes = df_features.loc[predict_indexes].dropna(how='all').index

# ----- Step 6: Training data -----
train_data = df_work[df_work['number_of_episodes_clean'].notna()]
X_train = train_data[features].copy()
X_train = X_train.replace(-1, np.nan).fillna(X_train.median())
y_train = train_data['number_of_episodes_clean']

# Prediction data
X_predict = df_features.loc[valid_predict_indexes].fillna(X_train.median())

# ----- Step 7: Random Forest model -----
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

# ----- Step 8: Predict -----
predicted_values = np.round(model.predict(X_predict)).astype(int)

# No 0 episodes – all 0s become 1
predicted_values[predicted_values < 1] = 1

# ----- Step 9: Insert prediction into column -----
df_work.loc[valid_predict_indexes, 'number_of_episodes_clean'] = predicted_values

# ----- Step 10: Test -----
print("✅ חיזוי הושלם בהצלחה!")
print("🔢 מספר ערכים שהושלמו:", len(predicted_values))
print(df_work['number_of_episodes_clean'].value_counts().sort_index())


# New df: df_work

In [ ]:
# Check df_work for 'number_of_episodes_clean' values

import matplotlib.pyplot as plt
import pandas as pd

# ----- Step 1: Counts -----
counts = df_work['number_of_episodes_clean'].value_counts().sort_index()

# ----- Step 2: Bar chart -----
plt.figure(figsize=(14,6))
plt.bar(counts.index, counts.values, color='skyblue')
plt.xlabel('Number of Episodes')
plt.ylabel('Count')
plt.title('Distribution of Number of Episodes (after filling unknowns)')
plt.xticks(counts.index, rotation=90)
plt.show()

# ----- Step 3: UNIQUE table with counts -----
unique_table = pd.DataFrame({
    'Number_of_Episodes': counts.index,
    'Count': counts.values
})

# Display the full table
print(unique_table)


In [ ]:
# Display the first 36 values in df_work 'number_of_episodes_clean'
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assume you have the column number_of_episodes_clean
episodes_cleaned = df_work['number_of_episodes_clean']

# Filter values ​​up to 36 (including NaN)
episodes_filtered = episodes_cleaned[ (episodes_cleaned <= 36) | (episodes_cleaned.isna()) ]

# Count the number of occurrences of each unique value including NaN
counts = episodes_filtered.value_counts(dropna=False).sort_index(key=lambda x: [float('-inf') if pd.isna(v) else v for v in x])

# Bar chart
plt.figure(figsize=(14,6))
plt.bar(counts.index.astype(str), counts.values, color='skyblue')  # הפיכה ל-string כדי שהNaN יוצג
plt.xlabel('Number of Episodes')
plt.ylabel('Count')
plt.title('Distribution of Number of Episodes (up to 36, including NaN)')
plt.xticks(rotation=45)
plt.show()

# Create a UNIQUE table with counts including NaN
unique_table_episodes = pd.DataFrame({
    'Number_of_Episodes': counts.index,
    'Count': counts.values
})

# Display the full table
print(unique_table_episodes)



In [ ]:
# compared to original df file - # Display the first 36 values ​​in df 'number_of_episodes'

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Assume you have the column number_of_episodes
episodes_cleaned = df['number_of_episodes']

# Filter values ​​up to 36 (including NaN)
episodes_filtered = episodes_cleaned[ (episodes_cleaned <= 36) | (episodes_cleaned.isna()) ]

# Count the number of occurrences of each unique value including NaN
counts = episodes_filtered.value_counts(dropna=False).sort_index(key=lambda x: [float('-inf') if pd.isna(v) else v for v in x])

# Bar chart
plt.figure(figsize=(14,6))
plt.bar(counts.index.astype(str), counts.values, color='skyblue')  # הפיכה ל-string כדי שהNaN יוצג
plt.xlabel('Number of Episodes')
plt.ylabel('Count')
plt.title('Distribution of Number of Episodes (up to 36, including NaN)')
plt.xticks(rotation=45)
plt.show()

# Create a UNIQUE table with counts including NaN
unique_table_episodes = pd.DataFrame({
    'Number_of_Episodes': counts.index,
    'Count': counts.values
})

# Display the full table
print(unique_table_episodes)


In [ ]:
# Check df_work
(df_work['number_of_episodes_clean'] == 0).sum()


In [ ]:
# To have an overview of missing data and types of data

summary = pd.DataFrame({
    'non_null_count': df_work.notnull().sum(),
    'null_percent': df_work.isnull().mean() * 100,
    'num_unique': df_work.nunique(),
    'data_type': df_work.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
# quantile(0.95) - check values ​​below to remove outliers later

# Variable to check

col = 'number_of_episodes_clean'

# Calculate the upper bound for the 95th percentile
upper_bound = df_work[col].quantile(0.95)
print(f"95th percentile upper bound for {col}: {upper_bound}")

# Filter the rows above the limit
df_work = df_work[df_work[col] <= upper_bound]

# Check how many rows are left
print(f"Number of rows after removing 95th percentile outliers: {len(df_work)}")


In [ ]:
# to remove seasons outliers!!!!!
# remove outliers here!!!!!!!!!!!
# remove 95% top outliers number_of_episodes_clean !!!!!!!!!
# Variable to test

col = 'number_of_episodes_clean'

# Calculate the upper bound for the 95th percentile
upper_bound = df_work[col].quantile(0.95)
print(f"95th percentile upper bound for {col}: {upper_bound}")

# Filter the rows above the limit
df_filtered = df_work[df_work[col] <= upper_bound]

# Output: Number of rows removed and percentage
num_removed = len(df_work) - len(df_filtered)
percent_removed = num_removed / len(df_work) * 100

print(f"Number of rows after removing 95th percentile outliers: {len(df_filtered)}")
print(f"Number of rows removed: {num_removed}")
print(f"Percent of dataset removed: {percent_removed:.2f}%")

# Save back to df_work
df_work = df_filtered


In [ ]:
# Delete original 'number_of_seasons' column - no need from here

df_work = df_work.drop(columns=['number_of_episodes'])

In [ ]:
df_work.head()

In [ ]:
df_work.info()

In [ ]:
# new df
df_clean = df_work.copy()

# New df name from here - df_clean

In [ ]:
# To have an overview of missing data and types of data

summary = pd.DataFrame({
    'non_null_count': df_clean.notnull().sum(),
    'null_percent': df_clean.isnull().mean() * 100,
    'num_unique': df_clean.nunique(),
    'data_type': df_clean.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

## Vote_count and vote_average

In [ ]:
# Uniqes count for 'vote_count'
df['vote_count'].value_counts().unique()

In [ ]:
# Better overlook of Uniqes count for 'vote_count'
# with Graphic display

import matplotlib.pyplot as plt
import pandas as pd

# Count unique values
vote_counts = df_clean['vote_count'].value_counts().sort_index()

# Create a full table
vote_counts_table = pd.DataFrame({
    'Vote_Count': vote_counts.index,
    'Frequency': vote_counts.values
})

# Display the full table
print(vote_counts_table)

# ----- Bar chart -----
# Filter values ​​up to 100 to make the graph readable

vote_counts_filtered = vote_counts[vote_counts.index <= 100]

plt.figure(figsize=(14,6))
plt.bar(vote_counts_filtered.index.astype(int), vote_counts_filtered.values, color='skyblue')
plt.xlabel('Vote Count')
plt.ylabel('Frequency')
plt.title('Distribution of vote_count (up to 100)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Better overlook of Uniqes count for 'vote_average'
# with Graphic display

import matplotlib.pyplot as plt
import pandas as pd

# Counting unique values
vote_average = df_clean['vote_average'].value_counts().sort_index()

# Creating a full table
vote_average_table = pd.DataFrame({
    'vote_average': vote_average.index,
    'Frequency': vote_average.values
})

# Displaying the full table
print(vote_average_table)

# ----- Bar graph -----
# Filtering values ​​up to 100 to make the graph readable

vote_average_filtered = vote_average[vote_average.index <= 100]

plt.figure(figsize=(14,6))
plt.bar(vote_average_filtered.index.astype(int), vote_average_filtered.values, color='skyblue')
plt.xlabel('Vote Count')
plt.ylabel('Frequency')
plt.title('Distribution of vote_average (up to 100)')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Compare values of 'vote_count' and 'vote_average'
df_clean[['vote_count', 'vote_average']].head()
df_clean[['vote_count', 'vote_average']].describe()

In [ ]:
# Check correlation
df_clean[['vote_count', 'vote_average']].corr()


In [ ]:
# Check correlation
import matplotlib.pyplot as plt

plt.scatter(df_clean['vote_count'], df_clean['vote_average'])
plt.xlabel('Vote Count')
plt.ylabel('Vote Average')
plt.title('Vote Count vs Vote Average')
plt.show()

In [ ]:
# Check correlation - log scale
plt.scatter(df_clean['vote_count'], df_clean['vote_average'])
plt.xscale('log')
plt.xlabel('Vote Count (log scale)')
plt.ylabel('Vote Average')
plt.title('Vote Count vs Vote Average (log scale)')
plt.show()


In [ ]:
# Check correlation - using bins

df_clean['vote_count_group'] = pd.cut(df_clean['vote_count'], bins=[0, 10, 100, 1000, 10000, 100000])
df_clean.groupby('vote_count_group')['vote_average'].mean()


In [ ]:
# Check Pearson correlation
import matplotlib.pyplot as plt
import seaborn as sns

# Scatterplot between vote_count and vote_average
plt.figure(figsize=(10,6))
sns.scatterplot(x='vote_count', y='vote_average', data=df_clean, alpha=0.3)

# Linear trendline
sns.regplot(x='vote_count', y='vote_average', data=df_clean, scatter=False, color='red', line_kws={"linewidth":2})

plt.xlabel('Vote Count')
plt.ylabel('Vote Average')
plt.title('Correlation between Vote Count and Vote Average')
plt.xlim(0, 1000)
plt.ylim(0, 10)
plt.show()

# Calculate correlation
correlation = df_clean['vote_count'].corr(df_clean['vote_average'])
print(f"Pearson correlation between vote_count and vote_average: {correlation:.4f}")


In [ ]:
# Create new column for 'vote_count' with bins - 'vote_count_group'
# Create a new binning column for vote_count

def vote_binning(v):
    if v == 0:
        return "No votes"
    elif v <= 10:
        return "Low"
    elif v <= 100:
        return "Medium"
    elif v <= 1000:
        return "High"
    else:
        return "Very High"

df_clean['vote_count_group'] = df_clean['vote_count'].apply(vote_binning)

# Check how many are in each group
print(df_clean['vote_count_group'].value_counts())


In [ ]:
df_clean.info()

In [ ]:
df_clean.head()

In [ ]:
# Create new column for 'vote_average' with bins - 'vote_average_group'

# Creating a binning column for vote_average

def vote_avg_binning(v):
    if v < 3:
        return "Very Low"
    elif v < 5:
        return "Low"
    elif v < 7:
        return "Medium"
    elif v < 8.5:
        return "High"
    else:
        return "Excellent"

df_clean['vote_average_group'] = df_clean['vote_average'].apply(vote_avg_binning)

# Check how many there are in each group
print(df_clean['vote_average_group'].value_counts())


In [ ]:
pd.crosstab(df_clean['vote_count_group'], df_clean['vote_average_group'])


In [ ]:
# check correlation between the two new columns
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10,6))
sns.heatmap(
    pd.crosstab(df_clean['vote_count_group'], df_clean['vote_average_group'], normalize='index'),
    annot=True, fmt=".2f", cmap="Blues"
)
plt.title("Vote Average Distribution by Vote Count Group")
plt.ylabel("Vote Count Group")
plt.xlabel("Vote Average Group")
plt.show()


In [ ]:
df_clean.info()

In [ ]:
# Check IQR outliers for vote_count
# vote_count outliers
# Variable to check

col = 'vote_count'

# Calculate Q1, Q3 and IQR
Q1 = df_work[col].quantile(0.25)
Q3 = df_work[col].quantile(0.75)
IQR = Q3 - Q1

# Outliers limits
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1 = {Q1}, Q3 = {Q3}, IQR = {IQR}")
print(f"Lower bound = {lower_bound}, Upper bound = {upper_bound}")

# Mark the rows that are Outliers
df_work['vote_count_outlier_iqr'] = (df_work[col] < lower_bound) | (df_work[col] > upper_bound)

# Summary
num_outliers = df_work['vote_count_outlier_iqr'].sum()
percent_outliers = num_outliers / len(df_work) * 100
print(f"Number of outliers in {col}: {num_outliers}")
print(f"Percent of dataset: {percent_outliers:.2f}%")


In [ ]:
# Check upper_percentile = 0.95 for 'vote_count'

# Variable to test

col = 'vote_count'

# Calculation of the upper limit for the 95th percentile
upper_percentile = 0.95
upper_bound = df_work[col].quantile(upper_percentile)
print(f"95th percentile upper bound for {col} = {upper_bound}")

# Marking the outliers according to the 95th percentile
df_work['vote_count_outlier_95'] = df_work[col] > upper_bound

# Summary
num_outliers = df_work['vote_count_outlier_95'].sum()
percent_outliers = num_outliers / len(df_work) * 100
print(f"Number of outliers (95th percentile) in {col}: {num_outliers}")
print(f"Percent of dataset: {percent_outliers:.2f}%")


In [ ]:
# to remove vote_count outliers!!!!!
# remove outliers here!!!!!!!!!!!
# remove 95% top outliers vote_count !!!!!!!!!
# Variable to test

col = 'vote_count'

# Calculate the upper bound for the 95th percentile
upper_bound = df_work[col].quantile(0.95)
print(f"95th percentile upper bound for {col}: {upper_bound}")

# Filter the rows above the limit
df_filtered = df_work[df_work[col] <= upper_bound]

# Output: Number of rows removed and percentage
num_removed = len(df_work) - len(df_filtered)
percent_removed = num_removed / len(df_work) * 100

print(f"Number of rows after removing 95th percentile outliers: {len(df_filtered)}")
print(f"Number of rows removed: {num_removed}")
print(f"Percent of dataset removed: {percent_removed:.2f}%")

# Save back to df_work
df_work = df_filtered


In [ ]:
# Check upper_percentile = 0.95 for 'vote_average'

# Variable to test
col = 'vote_average'

# # Setting a manual upper limit for Outliers
upper_bound = 9

# Mark Outliers
df_work['vote_average_outlier_95'] = df_work[col] > upper_bound

# Summary
num_outliers = df_work['vote_average_outlier_95'].sum()
percent_outliers = num_outliers / len(df_work) * 100

print(f"Upper bound for {col}: {upper_bound}")
print(f"Number of outliers in {col} (95th percentile / >9): {num_outliers}")
print(f"Percent of dataset: {percent_outliers:.2f}%")


In [ ]:
# to remove vote_average outliers!!!!!
# remove outliers here!!!!!!!!!!!
# remove 95% top outliers vote_average !!!!!!!!!
# Variable to test

col = 'vote_average'

# Calculate the upper bound for the 95th percentile
upper_bound = df_work[col].quantile(0.95)
print(f"95th percentile upper bound for {col}: {upper_bound}")

# Filter the rows above the limit
df_filtered = df_work[df_work[col] <= upper_bound]

# Output: Number of rows removed and percentage
num_removed = len(df_work) - len(df_filtered)
percent_removed = num_removed / len(df_work) * 100

print(f"Number of rows after removing 95th percentile outliers: {len(df_filtered)}")
print(f"Number of rows removed: {num_removed}")
print(f"Percent of dataset removed: {percent_removed:.2f}%")

# Save back to df_work
df_work = df_filtered


# explore episode_run_time

In [ ]:
# Check 'episode_run_time'

# Count the unique values ​​in the episode_run_time column

episode_run_time = df_clean['episode_run_time'].value_counts().sort_index()

# Create a complete table with all values
episode_run_time_table = pd.DataFrame({
    'episode_run_time': episode_run_time.index,
    'Frequency': episode_run_time.values
})

# Display the entire table
pd.set_option('display.max_rows', None)  # מגדיר להציג את כל השורות
print(episode_run_time_table)

In [ ]:
df_clean.info()

In [ ]:
# Data unknown and 0, -1 completion: Create new clean column 'episode_run_time_clean'
# Label encoding to 'type' and 'genres' columns - create 'type_encoded' and 'genres_clean' new columns

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder


# ----- Creating a clean target column -----
df_clean['episode_run_time_clean'] = df_clean['episode_run_time'].replace(0, np.nan)

# ----- LabelEncoding for column type -----
le_type = LabelEncoder()
df_clean['type_encoded'] = le_type.fit_transform(df_clean['type'])

# ----- Multi-Label Encoding for the genres column -----
df_clean['genres_clean'] = df_clean['genres'].apply(lambda x: x.split(',') if isinstance(x, str) else [])

# Build a genre dictionary
genre_to_id = {}
current_id = 0
for genres in df_clean['genres_clean']:
    for g in genres:
        if g not in genre_to_id:
            genre_to_id[g] = current_id
            current_id += 1

# Convert lists to average genres
df_clean['genres_encoded_avg'] = df_clean['genres_clean'].apply(
    lambda lst: np.mean([genre_to_id[g] for g in lst]) if len(lst) > 0 else np.nan
)

# ----- Choose predictor variables -----
features = ['number_of_seasons_clean', 'number_of_episodes_clean',
            'first_year', 'last_year', 'type_encoded', 'genres_encoded_avg']

# ----- Indexes for prediction (only where episode_run_time_clean is missing) -----
predict_indexes = df_clean[df_clean['episode_run_time_clean'].isna()].index

# ----- Training data -----
train_data = df_clean[df_clean['episode_run_time_clean'].notna()]
X_train = train_data[features].copy()
y_train = train_data['episode_run_time_clean']

# ----- Prediction data -----
# Here we temporarily fill NaN only inside X_predict, not in the DF itself
X_predict = df_clean.loc[predict_indexes, features].copy()
X_predict_filled = X_predict.fillna(X_train.median())

# ----- RandomForest model -----
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train.fillna(X_train.median()), y_train)

# ----- Prediction -----
predicted_values = np.round(model.predict(X_predict_filled)).astype(int)
predicted_values[predicted_values < 1] = 1  # אין 0 דקות

# ----- Insert prediction into target column only -----
df_clean.loc[predict_indexes, 'episode_run_time_clean'] = predicted_values

# ----- Check -----
print("✅ חיזוי episode_run_time_clean הושלם!")
print("🔢 מספר ערכים שהושלמו:", len(predicted_values))
print(df_clean['episode_run_time_clean'].isna().sum())



In [ ]:
df_clean.info()

In [ ]:
# Compare bettwen before anf after prediction

import matplotlib.pyplot as plt

# Distribution before completion (0 still exist)
plt.figure(figsize=(16,6))
plt.subplot(1,2,1)
(df['episode_run_time']
 .value_counts()
 .sort_index()
 .plot(kind='bar', color='skyblue'))
plt.title("התפלגות episode_run_time - לפני ההשלמה")
plt.xlabel("Runtime (דקות)")
plt.ylabel("מספר סדרות")

# Distribution after completion
plt.subplot(1,2,2)
(df_clean['episode_run_time_clean']
 .value_counts()
 .sort_index()
 .plot(kind='bar', color='orange'))
plt.title("התפלגות episode_run_time_clean - אחרי ההשלמה")
plt.xlabel("Runtime (דקות)")
plt.ylabel("מספר סדרות")

plt.tight_layout()
plt.show()


In [ ]:
df_clean.head()

In [ ]:
# High resolution of episode_run_time_clean values ​​- top 200 values
import matplotlib.pyplot as plt

# Take the first 200 values ​​by runtime (sorted by index or value)
top_200 = df_clean['episode_run_time_clean'].value_counts().sort_index().head(200)

plt.figure(figsize=(16,6))
top_200.plot(kind='bar', color='orange')
plt.title("התפלגות episode_run_time_clean - 200 הראשונים אחרי ההשלמה")
plt.xlabel("Runtime (דקות)")
plt.ylabel("מספר סדרות")
plt.xticks(rotation=90)
plt.show()


In [ ]:
# Unique values ​​in episode_run_time_clean

# Count unique values ​​in episode_run_time column

episode_run_time_clean = df_clean['episode_run_time_clean'].value_counts().sort_index()

# Create a complete table with all values
episode_run_time_table = pd.DataFrame({
    'episode_run_time_clean': episode_run_time_clean.index,
    'Frequency': episode_run_time_clean.values
})

# Display the entire table
pd.set_option('display.max_rows', None)
print(episode_run_time_table)

In [ ]:
# Count the unique values ​​in the episode_run_time column
episode_run_time = df_clean['episode_run_time'].value_counts().sort_index()

# Create a complete table with all the values
episode_run_time_table = pd.DataFrame({
    'episode_run_time': episode_run_time.index,
    'Frequency': episode_run_time.values
})

# Display the entire table
pd.set_option('display.max_rows', None)
print(episode_run_time_table)

In [ ]:
# All unique values ​​in the 'genres_encoded_avg' column

unique_values = df_clean['genres_encoded_avg'].unique()
print(unique_values)
print("מספר ערכים ייחודיים:", len(unique_values))


In [ ]:
# IQR analysis for 'episode_run_time_clean'
col = 'episode_run_time_clean'

# Calculation of Q1, Q3 and the IQR
Q1 = df_clean[col].quantile(0.25)
Q3 = df_clean[col].quantile(0.75)
IQR = Q3 - Q1

# Boundaries
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Mark Outliers
df_clean['episode_run_time_outlier_iqr'] = (df_clean[col] < lower_bound) | (df_clean[col] > upper_bound)

# Summary
num_outliers_iqr = df_clean['episode_run_time_outlier_iqr'].sum()
percent_outliers_iqr = num_outliers_iqr / len(df_clean) * 100

print(f"IQR bounds: {lower_bound} – {upper_bound}")
print(f"Number of outliers (IQR): {num_outliers_iqr}")
print(f"Percent of dataset: {percent_outliers_iqr:.2f}%")


In [ ]:
# to remove episode_run_time_clean outliers!!!!!
# remove outliers here!!!!!!!!!!!
# remove IQR top outliers episode_run_time_clean !!!!!!!!!
# Variable to test

col = 'episode_run_time_clean'

# Calculate Q1, Q3 and IQR
Q1 = df_clean[col].quantile(0.25)
Q3 = df_clean[col].quantile(0.75)
IQR = Q3 - Q1

# Bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"IQR bounds: {lower_bound} – {upper_bound}")

# Filter out rows that are not Outliers
df_clean = df_clean[(df_clean[col] >= lower_bound) & (df_clean[col] <= upper_bound)]

# Check how many rows are left
num_rows = len(df_clean)
print(f"Number of rows after removing IQR outliers: {num_rows}")


In [ ]:
# Delete original column 'episode_run_time' - no need for it anymore

df_clean = df_clean.drop(columns=['episode_run_time'])

In [ ]:
# Remove 'genres' columns - no need anymore

df_clean = df_clean.drop(columns=['genres'])

# Continue outliers

## After removing outliers above - this section will check if a second outlier removal is needed after Log transformation

In [ ]:
# Check numeric columns for more deep outlier analysis

numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns
print(numeric_cols)

In [ ]:
# Skewness check
skew_values = df_clean[numeric_cols].skew().sort_values(ascending=False)
print(skew_values)

In [ ]:
# Before Log transformation for numeric columns - Check values ​​under -1 or = -1.
# Important for Log transformation

import numpy as np

# Numeric columns that fit the log, without target and id
log_cols_features = [
    'number_of_episodes_clean',
    'episode_run_time_clean',
    'number_of_seasons_clean',
    'vote_count'
]


(df_clean[log_cols_features] <= -1).sum()

In [ ]:
# Log transformation for these columns

import numpy as np

log_cols_features = [
    'number_of_episodes_clean',
    'episode_run_time_clean',
    'number_of_seasons_clean',
    'vote_count'
]

continuous_df_log = np.log1p(df_clean[log_cols_features])

In [ ]:
# Check IQR - this time after Log transformation

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp

# ---- 1️⃣ Setting up the columns for log and preparing ----
log_cols_features = [
    'number_of_episodes_clean',
    'episode_run_time_clean',
    'number_of_seasons_clean',
    'vote_count'
]

# Handling negative or too small values
df_clean[log_cols_features] = df_clean[log_cols_features].clip(lower=0)

# Creating a log version
continuous_df_log = np.log1p(df_clean[log_cols_features])

# ---- 2️⃣ Function to detect outliers by IQR ----
def treat_outliers(df, features):
    outliers_df = pd.DataFrame(index=df.index)
    summary = {}

    for feature in features:
        Q1 = df[feature].quantile(0.25)
        Q3 = df[feature].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = ((df[feature] < lower_bound) | (df[feature] > upper_bound))
        outliers_df[feature] = outliers.astype(int)
        summary[feature] = {
            "lower_bound": lower_bound,
            "upper_bound": upper_bound,
            "n_outliers": outliers.sum(),
            "percent_outliers": (outliers.sum() / len(df)) * 100
        }

    summary_df = pd.DataFrame(summary).T.sort_values(by="percent_outliers", ascending=False)
    return outliers_df, summary_df

# ---- 3️⃣ identifying outliers ----
outliers_df, summary_df = treat_outliers(continuous_df_log, continuous_df_log.columns)
display(summary_df)

# ---- 4️⃣ cut of exceptions according to the 95th percentile ----
for feature in continuous_df_log.columns:
    cap = continuous_df_log[feature].quantile(0.95)
    continuous_df_log[feature] = continuous_df_log[feature].clip(upper=cap)

# ---- 5️⃣ graph top 5 features with the most outliers ----
plt.figure(figsize=(10,5))
sns.barplot(
    x=summary_df.index[:5],
    y=summary_df["percent_outliers"][:5]
)
plt.xticks(rotation=45)
plt.title("Top 5 Features with Highest Percent of Outliers")
plt.ylabel("Percent of Outliers")
plt.show()

# ---- 6️⃣ Testing of change in distribution and correlation with Target ----
out_df = pd.DataFrame()

for col in outliers_df:
    out = continuous_df_log[col]
    non_out = continuous_df_log[col][outliers_df[col] == 0]
    cor_out = df_clean['popularity']
    cor_non_out = df_clean['popularity'][outliers_df[col] == 0]

    # Filter out NaN values for correlation calculation
    valid_out = ~np.isnan(out) & ~np.isnan(cor_out)
    valid_non_out = ~np.isnan(non_out) & ~np.isnan(cor_non_out)

    if valid_out.sum() > 1 and valid_non_out.sum() > 1:
        cor_change = '+' if abs(np.corrcoef(out[valid_out], cor_out[valid_out])[0, 1] -
                                np.corrcoef(non_out[valid_non_out], cor_non_out[valid_non_out])[0, 1]) > 0.05 else '-'
    else:
        cor_change = '-'

    dist_change = '+' if ks_2samp(out.dropna(), non_out.dropna())[1] < 0.05 else '-'

    new_row = pd.DataFrame({'feature': [col],
                            'outliers_cnt': [outliers_df[col].sum()],
                            'distribution_changed': [dist_change],
                            'correlation_changed': [cor_change]})

    out_df = pd.concat([out_df, new_row], ignore_index=True)

display(out_df)


In [ ]:
# Add recommendation to remove or not outliers values
# Add recommendation to consider drop


out_df['drop'] = np.where(
    (out_df['distribution_changed'] == '+') & (out_df['correlation_changed'] == '-'),
    'yes',
    'no'
)

# List of features to consider dropping or handling
to_drop = out_df[out_df['drop'] == 'yes']['feature']

# Display DataFrame
display(out_df)


In [ ]:
# Change to tuples - 'tuples' column

# Convert lists or tuples to strings
for col in ['genres_clean']:
    df_clean[col] = df_clean[col].apply(lambda x: str(x) if isinstance(x, (list, tuple)) else x)

# Now you can convert to category without error
categorical_cols = [
    'type', 'status', 'networks', 'production_countries',
    'original_language', 'vote_count_group', 'vote_average_group',
    'type_encoded', 'genres_clean'
]

for col in categorical_cols:
    df_clean[col] = df_clean[col].astype('category')

In [ ]:
# Deal outliers with CAPPING and show before and after graph

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ---- 0️⃣ Convert categorical columns to category ----
categorical_cols = [
    'status', 'networks',
    'original_language', 'vote_count_group', 'vote_average_group',
    'type_encoded', 'genres_clean'
]

for col in categorical_cols:
    df_clean[col] = df_clean[col].astype('category')

# ---- 1️⃣ Set columns to log ----
log_cols_features = [
    'number_of_episodes_clean',
    'episode_run_time_clean',
    'number_of_seasons_clean',
    'vote_count'
]

# Handling negative or small values Too
df_clean[log_cols_features] = df_clean[log_cols_features].clip(lower=0)

# Create initial log version
continuous_df_log = np.log1p(df_clean[log_cols_features])

# ---- 2️⃣ List of features to be treated (drop == 'yes') ----
features_to_cap = out_df[out_df['drop'] == 'yes']['feature'].tolist()

# ---- 3️⃣ Save log version before Capping ----
continuous_df_log_before = continuous_df_log.copy()

# ---- 4️⃣ Cut to 95th percentile for these features ----
for feature in features_to_cap:
    cap = continuous_df_log[feature].quantile(0.95)
    continuous_df_log[feature] = continuous_df_log[feature].clip(upper=cap)

# ---- 5️⃣ Comparison graph before and after ----
for feature in features_to_cap:
    plt.figure(figsize=(10,4))
    sns.kdeplot(continuous_df_log_before[feature], label='Before capping', fill=True)
    sns.kdeplot(continuous_df_log[feature], label='After capping', fill=True)
    plt.title(f'Distribution of {feature} Before/After Capping')
    plt.xlabel(feature)
    plt.ylabel('Density')
    plt.legend()
    plt.show()

# ---- 6️⃣ final result ----
print("Features capped at 95th percentile:", features_to_cap)
display(continuous_df_log.head())


In [ ]:
# Overall view after and before CAPPING - outliers

import matplotlib.pyplot as plt
import seaborn as sns

# List of all the features for the plot
all_features = ['number_of_episodes_clean', 'episode_run_time_clean',
                'number_of_seasons_clean', 'vote_count']

plt.figure(figsize=(15,10))

for feature in all_features:
    sns.kdeplot(continuous_df_log_before[feature], label=f'{feature} Before', linestyle='--')
    sns.kdeplot(continuous_df_log[feature], label=f'{feature} After', linestyle='-')

plt.title("Comparison of Distributions Before/After Capping for All Features")
plt.xlabel("Log1p Values")
plt.ylabel("Density")
plt.legend()
plt.show()


In [ ]:
df_clean.info()

In [ ]:
df_clean.head()

# Target Encoding

In [ ]:
summary = pd.DataFrame({
    'non_null_count': df_clean.notnull().sum(),
    'null_percent': df_clean.isnull().mean() * 100,
    'num_unique': df_clean.nunique(),
    'data_type': df_clean.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
# Label encoding to continue analysis
from sklearn.preprocessing import LabelEncoder

# --- 1️⃣ Clean copy of the DataFrame ---
df_final = df_clean.copy()

# --- 2️⃣ Keep numeric features as they are (or with log if you wanted) ---
numeric_features_continuous = list(continuous_df_log.columns)
for col in numeric_features_continuous:
    df_final[col] = continuous_df_log[col]

# --- 3️⃣ categorical features with Label Encoding ---
categorical_cols = [
    'type', 'status', 'genres_clean', 'networks',
    'original_language', 'vote_count_group', 'vote_average_group',
    'production_countries'
]

for col in categorical_cols:
    le = LabelEncoder()
    df_final[col] = le.fit_transform(df_final[col].astype(str))

# --- 4️⃣ Boolean columns remain as Bool ---
bool_cols = ['adult', 'in_production', 'year_known']

# --- 5️⃣ list of feature sets ---
numeric_features = numeric_features_continuous + ['vote_average', 'first_year', 'last_year',
                                                  'genres_encoded_avg', 'type_encoded']
categorical_features = categorical_cols + bool_cols
target = 'popularity'

# --- 6️⃣ test ---
print("Columns ready for modeling:")
display(df_final.head())

print("Numeric features:", numeric_features)
print("Categorical features (Label Encoded + Bool):", categorical_features)
print("Target:", target)


In [ ]:
summary = pd.DataFrame({
    'non_null_count': df_final.notnull().sum(),
    'null_percent': df_final.isnull().mean() * 100,
    'num_unique': df_final.nunique(),
    'data_type': df_final.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

# Check correlation again - for all current columns


In [ ]:
# Correlation with 'popularity' - the target column
import seaborn as sns
import matplotlib.pyplot as plt

# ---- 1️⃣ select only numeric columns including target ----
numeric_cols = df_clean.select_dtypes(include=['float64', 'int64']).columns.tolist()

# ---- 2️⃣ calculate correlation ----
corr_matrix = df_clean[numeric_cols].corr()

# ---- 3️⃣ remove only the line of popularity ----
popularity_corr = corr_matrix['popularity'].sort_values(ascending=False)

print("Correlation of all numeric features with popularity:")
display(popularity_corr)

# ---- 4️⃣ heatmap of correlation with popularity ----
plt.figure(figsize=(10,6))
sns.heatmap(popularity_corr.to_frame(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title("Correlation with Popularity")
plt.show()


In [ ]:
# deal with tuples before analyzing important words for popularity score

# Identifies columns that have lists or tuples

list_cols = [col for col in df_clean.columns
             if df_clean[col].apply(lambda x: isinstance(x, (list, tuple))).any()]

print("Columns with list/tuple values:", list_cols)

# Convert each list/tuple to a string (comma-separated)
for col in list_cols:
    df_clean[col] = df_clean[col].apply(lambda x: ','.join(x) if isinstance(x, (list, tuple)) else str(x))

# Now you can check number of unique safely
unique_counts = df_clean.nunique()
print(unique_counts)


In [ ]:
# Cosine Similarity - for text 'overview' and 'final_name' - to understand words that effect popularity
# Tow new columns: 'final_name_score' and 'overview_score'

import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Parameters
TOP_N = 10000
BATCH_SIZE = 5000

# Select Top-N by popularity
top_df = df_final.nlargest(TOP_N, 'popularity')
top_final_names = top_df['final_name'].fillna('').values
top_overviews = top_df['overview'].fillna('').values
top_popularity = top_df['popularity'].values

# TF-IDF on words only
tfidf_vectorizer_name = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_vectorizer_overview = TfidfVectorizer(stop_words='english', max_features=5000)

tfidf_vectorizer_name.fit(top_final_names)
tfidf_vectorizer_overview.fit(top_overviews)

# Transform Top-N
top_final_names_tfidf = tfidf_vectorizer_name.transform(top_final_names)
top_overviews_tfidf = tfidf_vectorizer_overview.transform(top_overviews)

# A function to calculate a grade in a batch
def compute_score_batch(text_series, tfidf_vectorizer, top_tfidf, top_popularity):
    scores = np.zeros(len(text_series))
    for start in range(0, len(text_series), BATCH_SIZE):
        end = min(start + BATCH_SIZE, len(text_series))
        batch_text = text_series[start:end].fillna('').values
        batch_tfidf = tfidf_vectorizer.transform(batch_text)
        cos_sim = cosine_similarity(batch_tfidf, top_tfidf)
        scores[start:end] = np.dot(cos_sim, top_popularity) / (cos_sim.sum(axis=1) + 1e-6)
    return scores

# Calculation of grades
df_final['final_name_score'] = compute_score_batch(df_final['final_name'], tfidf_vectorizer_name, top_final_names_tfidf, top_popularity)
df_final['overview_score'] = compute_score_batch(df_final['overview'], tfidf_vectorizer_overview, top_overviews_tfidf, top_popularity)

# test
print(df_final[['final_name', 'final_name_score', 'overview_score', 'popularity']].head())


# New df: df_final

In [ ]:
summary = pd.DataFrame({
    'non_null_count': df_final.notnull().sum(),
    'null_percent': df_final.isnull().mean() * 100,
    'num_unique': df_final.nunique(),
    'data_type': df_final.dtypes,
}).sort_values(by='null_percent', ascending=False)

print(summary)

In [ ]:
df_final.head()

In [ ]:
df_final.describe()


# Continue correlation and statistics

In [ ]:
# Check statistics related to 'popularity'

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, f_oneway, pearsonr
from pandas.api.types import is_numeric_dtype

target = 'popularity'
results = []

for col in df_final.columns:
    if col == target:
        continue

    # 1️⃣ boolean columns → t-test
    if df_final[col].dtype == 'bool':
        groups = df_final[col].unique()
        if len(groups) == 2:
            group1 = df_final[df_final[col] == groups[0]][target]
            group2 = df_final[df_final[col] == groups[1]][target]
            if len(group1) > 1 and len(group2) > 1:
                stat, pval = ttest_ind(group1, group2, equal_var=False, nan_policy='omit')
                results.append({'feature': col, 'test': 't-test', 'p_value': pval})

    # 2️⃣  Categorical columns → ANOVA limited to ≤10 categories
    elif df_final[col].dtype == 'object' or str(df_final[col].dtype).startswith('category'):
        n_unique = df_final[col].nunique()
        if 1 < n_unique <= 10:
            groups = [df_final[df_final[col] == val][target]
                      for val in df_final[col].unique()
                      if len(df_final[df_final[col] == val]) > 1]
            if len(groups) > 1:
                stat, pval = f_oneway(*groups)
                results.append({'feature': col, 'test': 'ANOVA', 'p_value': pval})

    # 3️⃣ Numeric columns → Pearson correlation
    elif is_numeric_dtype(df_final[col]):
        if df_final[col].nunique() > 1:
            corr, pval = pearsonr(df_final[col], df_final[target])
            results.append({'feature': col, 'test': 'Correlation', 'p_value': pval, 'corr': corr})

# Create summary table
results_df = pd.DataFrame(results)
results_df['significant'] = results_df['p_value'] < 0.05

# Sort by significance
results_df = results_df.sort_values('p_value')

# Display sorted results
pd.set_option('display.float_format', '{:.3e}'.format)
display(results_df)


In [ ]:
# Check statistic related to 'popularity'

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

# Select all numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns

# Calculate correlation with target
corr_matrix = df[numeric_cols].corr()
pop_corr = corr_matrix['popularity'].drop('popularity')

# Create DataFrame sorted by absolute value of correlation
corr_df = pd.DataFrame({
    'feature': pop_corr.index,
    'corr': pop_corr.values
}).sort_values(by='corr', key=abs, ascending=False)

# Add a column that defines color by positive/negative
corr_df['color'] = corr_df['corr'].apply(lambda x: 'green' if x > 0 else 'red')

# Bar graph
plt.figure(figsize=(10,8))
sns.barplot(
    x='corr',
    y='feature',
    data=corr_df,
    hue='color',
    dodge=False,
    palette={'green':'green','red':'red'},
    legend=False
)
plt.axvline(0, color='black', linestyle='--')
plt.title('Correlation of All Numeric Features with Popularity')
plt.xlabel('Pearson Correlation')
plt.ylabel('Feature')

# Add numeric value next to bar
for index, row in corr_df.iterrows():
    plt.text(row['corr'], index, f"{row['corr']:.2f}", color='black', va='center')

plt.tight_layout()
plt.show()


In [ ]:
# Check statistic related to 'popularity'

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Suppose your DataFrame is called df_final and includes the column 'popularity''

# 1️⃣ Select only numeric columns
numeric_cols = df_final.select_dtypes(include='number').columns.tolist()

# 2️⃣ Calculate correlation of each numeric column with 'popularity'
corr_dict = {}
for col in numeric_cols:
    if col != 'popularity':
        corr_dict[col] = df_final[col].corr(df_final['popularity'])

# 3️⃣ Convert to DataFrame
top_corr = pd.DataFrame({
    'feature': list(corr_dict.keys()),
    'corr': list(corr_dict.values())
})

# 4️⃣ Take the 15 values Top by absolute value of correlation
top_corr = top_corr.reindex(top_corr['corr'].abs().sort_values(ascending=False).index).head(15)

# 5️⃣ Graph
plt.figure(figsize=(10,6))
sns.barplot(
    x='corr',
    y='feature',
    data=top_corr,
    hue='feature',
    dodge=False,
    palette='coolwarm',
    legend=False
)
plt.axvline(0, color='black', linestyle='--')
plt.title('Top 15 Numeric Features Correlation with Popularity')
plt.xlabel('Pearson Correlation')
plt.ylabel('Feature')
plt.tight_layout()
plt.show()


In [ ]:
# Check statistic related to 'popularity'

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Copy of results_df
stats_df = results_df.copy()

# For numeric variables, also consider direction (corr)
stats_df['effect'] = stats_df.apply(
    lambda row: row['corr'] if row['test']=='Correlation' else np.nan, axis=1
)

#  Consider -log10(p_value) to highlight significance in the graph
stats_df['neg_log_p'] = -np.log10(stats_df['p_value'])

# Sort by significance
stats_df = stats_df.sort_values('neg_log_p', ascending=False)

# Display Table
display(stats_df[['feature','test','effect','p_value','neg_log_p','significant']])

# Graph of top 20 by significance
top20 = stats_df.head(20)

plt.figure(figsize=(10,6))
sns.barplot(
    x='neg_log_p',
    y='feature',
    data=top20,
    hue='effect',
    dodge=False,
    palette='coolwarm',
    legend=False
)
plt.axvline(-np.log10(0.05), color='black', linestyle='--', label='p=0.05')
plt.xlabel('-log10(p-value)')
plt.ylabel('Feature')
plt.title('Top 20 Features by Significance for Popularity')
plt.tight_layout()
plt.show()


In [ ]:

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind, f_oneway, pearsonr, chi2_contingency
from pandas.api.types import is_numeric_dtype, is_bool_dtype
import matplotlib.pyplot as plt
import seaborn as sns

target = 'popularity'



In [ ]:
# statistics and correlation

results_regular = []

for col in df_final.columns:
    if col == target:
        continue

    # Boolean → t-test
    if is_bool_dtype(df_final[col]):
        groups = df_final[col].unique()
        if len(groups) == 2:
            g1 = df_final[df_final[col] == groups[0]][target]
            g2 = df_final[df_final[col] == groups[1]][target]
            if len(g1) > 1 and len(g2) > 1:
                stat, pval = ttest_ind(g1, g2, equal_var=False, nan_policy='omit')
                results_regular.append({'feature': col, 'test':'t-test', 'p_value':pval, 'effect_size':np.nan})

    # small categoricals → ANOVA (≤10 values)
    elif df_final[col].dtype == 'object' or str(df_final[col].dtype).startswith('category'):
        n_unique = df_final[col].nunique()
        if 1 < n_unique <= 10:
            groups = [df_final[df_final[col] == val][target]
                      for val in df_final[col].unique()
                      if len(df_final[df_final[col] == val]) > 1]
            if len(groups) > 1:
                stat, pval = f_oneway(*groups)
                results_regular.append({'feature': col, 'test':'ANOVA', 'p_value':pval, 'effect_size':np.nan})

    #  Pearson correlation
    elif is_numeric_dtype(df_final[col]):
        if df_final[col].nunique() > 1:
            corr, pval = pearsonr(df_final[col], df_final[target])
            results_regular.append({'feature': col, 'test':'Correlation', 'p_value':pval, 'effect_size':corr})

df_regular = pd.DataFrame(results_regular)
df_regular['significant'] = df_regular['p_value'] < 0.05
epsilon = 1e-300
df_regular['neg_log_p'] = -np.log10(df_regular['p_value'].clip(lower=epsilon))
df_regular = df_regular.sort_values('neg_log_p', ascending=False)


In [ ]:
def cramers_v(x, y):
    """חישוב Cramér's V למשתנים קטגוריאליים"""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k-1, r-1))

large_cats = [col for col in df_final.columns
              if (df_final[col].dtype=='object' or str(df_final[col].dtype).startswith('category'))
              and df_final[col].nunique() > 10]

results_large = []

for col in large_cats:
    try:
        # Use qcut to divide target into 10 groups
        cv = cramers_v(df_final[col], pd.qcut(df_final[target], q=10, duplicates='drop'))
        results_large.append({'feature':col,'Cramers_V':cv})
    except:
        continue

df_large = pd.DataFrame(results_large)
df_large = df_large.sort_values('Cramers_V', ascending=False)


In [ ]:
# Show for popularity the top 20 Regular Features by Significance
top20_regular = df_regular.head(20)

plt.figure(figsize=(10,6))
sns.barplot(
    x='neg_log_p',
    y='feature',
    data=top20_regular,
    hue='effect_size',
    dodge=False,
    palette='coolwarm',
    legend=False
)
plt.axvline(-np.log10(0.05), color='black', linestyle='--', label='p=0.05')
plt.xlabel('-log10(p-value)')
plt.ylabel('Feature')
plt.title('Top 20 Regular Features by Significance')
plt.tight_layout()
plt.show()


In [ ]:
# Summary of the columns above

# Select the columns that summarize the graph

summary_table = df_regular[['feature','test','effect_size','p_value','neg_log_p','significant']]

# Sort by significance -log10(p-value)
summary_table = summary_table.sort_values('neg_log_p', ascending=False)

# Display the table
display(summary_table)


In [ ]:
top20_large = df_large.head(20)

plt.figure(figsize=(10,6))
sns.barplot(
    x='Cramers_V',
    y='feature',
    data=top20_large,
    palette='viridis',
    hue='feature',
    dodge=False,
    legend=False
)
plt.xlabel("Cramér's V")
plt.ylabel('Feature')
plt.title('Top 20 Large Categorical Features by Cramér\'s V')
plt.tight_layout()
plt.show()


In [ ]:
# Summary of two different types of statistical analyses

# First we will take care of regular variables (already have df_regular)

df_regular_summary = df_regular.copy()
df_regular_summary.rename(columns={'effect_size':'measure'}, inplace=True)
df_regular_summary['type'] = 'regular'  # לסמן סוג משתנה

# Now large variables (Cramér’s V)
df_large_summary = df_large.copy()
df_large_summary.rename(columns={'Cramers_V':'measure'}, inplace=True)
df_large_summary['p_value'] = np.nan
df_large_summary['neg_log_p'] = np.nan
df_large_summary['significant'] = np.nan
df_large_summary['test'] = 'Cramers_V'
df_large_summary['type'] = 'large_categorical'

# Adjust columns to merge
df_large_summary = df_large_summary[['feature','test','measure','p_value','neg_log_p','significant','type']]

df_regular_summary = df_regular_summary[['feature','test','measure','p_value','neg_log_p','significant','type']]

# Merge the two dataframes
df_summary_all = pd.concat([df_regular_summary, df_large_summary], ignore_index=True)

# Sort by measure or neg_log_p (for regular variables)
df_summary_all.sort_values(by=['type','neg_log_p','measure'], ascending=[True, False, False], inplace=True)

# Display The table
display(df_summary_all)


In [ ]:
# Saving DataFrame as CSV file

df_final.to_csv('df_final_last_GitHub.csv', index=False)
print("Saved df_final to 'df_final_last_GitHub.csv'")

In [ ]:
from google.colab import files
files.download('df_final_last_GitHub.csv')

In [ ]:
# For GitHub upload
from google.colab import drive
drive.mount('/content/drive')

In [3]:
import nbformat

# Path ל-notebook הנוכחי
path = '/content/drive/MyDrive/Project TV show popularity/advance project/More advanced project/For GitHub/Upload to GitHub/GitHub_2_TV_show_popularity_part_two_upload.ipynb'  # שנה לפי הנתיב שלך


# קריאה ועריכה של ה-notebook
nb = nbformat.read(path, as_version=4)

# ניקוי metadata בעייתית
if "widgets" in nb.metadata:
    del nb.metadata["widgets"]
if "colab" in nb.metadata:
    del nb.metadata["colab"]
if "celltoolbar" in nb.metadata:
    del nb.metadata["celltoolbar"]

# שמירה מחדש
nbformat.write(nb, path)
print("✅ Notebook cleaned and ready for GitHub!")

✅ Notebook cleaned and ready for GitHub!
